In [ ]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import openai

# 1. Pinecone 초기화 및 LangChain 설정
pinecone.init(api_key="your-pinecone-api-key", environment="us-west1-gcp")
index_name = "travel-recommendations"

# 2. OpenAI API 키 설정 및 임베딩 모델 초기화
openai.api_key = "your-openai-api-key"
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
index = Pinecone.from_existing_index(index_name, embeddings)

# 3. 검색 함수 정의 (여행 스타일 및 동행인 기반)
def search_places(city, companions, travel_style, top_k=10):
    query = f"Best places in {city} for {companions} with focus on {travel_style}."
    results = index.similarity_search(query, top_k=top_k)
    return results

# 4. 여행 일정 생성을 위한 프롬프트 템플릿
prompt_template = """
당신은 여행 일정 AI 전문가입니다. 아래의 정보를 바탕으로 {city}에서 {trip_duration} 동안의 여행 일정을 생성하세요.

- 여행 날짜: {travel_dates}
- 동행인: {companions}
- 여행 스타일: {travel_style}
- 추천 장소:
{recommendations}

**제약 조건**:
- 오전, 오후, 저녁 일정으로 구분하세요.
- 각 일정에 맞는 관광지와 맛집을 포함하세요.
- 운영 시간과 주소를 포함하세요.
- 동일한 장소를 중복하지 마세요.
- 동선을 최적화해 이동 시간을 최소화하세요.
- 일정은 한국어로 작성해주세요.
"""

# 5. 프롬프트와 LLMChain 설정
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4"
)

def generate_itinerary(city, trip_duration, travel_dates, companions, travel_style, recommendations):
    formatted_prompt = prompt_template.format(
        city=city,
        trip_duration=trip_duration,
        travel_dates=travel_dates,
        companions=companions,
        travel_style=travel_style,
        recommendations=recommendations
    )

    # LangChain 프롬프트 구성
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template("You are a travel itinerary expert."),
            HumanMessagePromptTemplate.from_template(formatted_prompt)
        ]
    )

    # LLMChain 설정 및 실행
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True
    )

    # 결과 생성
    result = conversation.run({})
    return result

# 6. 메인 함수: 사용자 입력 및 검색 실행
def main():
    # 사용자 입력 예시
    itinerary_details = {
        "city": "Paris",
        "trip_duration": "3박 4일",
        "travel_dates": "2024-11-15 ~ 2024-11-18",
        "companions": "친구와",
        "travel_style": "쇼핑, 관광지, 맛집"
    }

    # 장소 검색 실행
    search_results = search_places(
        city=itinerary_details["city"],
        companions=itinerary_details["companions"],
        travel_style=itinerary_details["travel_style"],
        top_k=20  # 최대 20개 장소를 가져옴
    )

    # 추천 장소 리스트 구성
    recommendations = "\\n".join([
        f"- {match.metadata['name']} (Category: {match.metadata['category']})"
        for match in search_results
    ])

    # 여행 일정 생성 호출
    itinerary = generate_itinerary(
        city=itinerary_details["city"],
        trip_duration=itinerary_details["trip_duration"],
        travel_dates=itinerary_details["travel_dates"],
        companions=itinerary_details["companions"],
        travel_style=itinerary_details["travel_style"],
        recommendations=recommendations
    )

    # 결과 출력
    print(itinerary)

# 7. 코드 실행
if __name__ == "__main__":
    main()
